### Main points

#### Group analysis across ROI

- Make average and sem of pRF size for fovea and periphery of controls / amblyo / aniso / strab / mixed
- Make average and sem of pRF size for fovea and periphery of controls / amblyo / aniso / strab / mixed
- Make difference of control with amblyo / aniso / strab / mixed of pRF size for fovea and periphery
- Make difference of control with / amblyo / aniso / strab / mixed of pRF CM for fovea and periphery
- Test significance of control vs. patient of pRF size for fovea and periphery
- Test significance of control vs. patient of pRF CM for fovea and periphery
- Test significance of fovea vs. periphery of pRF size
- Test significance of fovea vs. periphery of pRF CM
  

In [1]:
# General imports
import os
import sys
import json
import glob
import ipdb
import pandas as pd
import numpy as np
deb = ipdb.set_trace

In [138]:
# Inputs
data_dir = '/home/mszinte/disks/meso_S/data'
code_dir = '/home/mszinte/disks/meso_H/projects'
amblyo_dir = 'amblyo_prf'
control_dir = 'RetinoMaps'

# Personal import
sys.path.append("{}/{}/analysis_code/utils".format(code_dir, amblyo_dir))
from plot_utils import *

# load amblyo settings
with open('{}/{}/analysis_code/settings.json'.format(code_dir, amblyo_dir)) as f:
    json_s = f.read()
    amblyo_analysis_info = json.loads(json_s)
amblyo_subjects = amblyo_analysis_info['subjects_patient']
strab_subjects = amblyo_analysis_info['subjects_strab']
aniso_subjects = amblyo_analysis_info['subjects_aniso']
mixed_subjects = amblyo_analysis_info['subjects_mixed']
formats = amblyo_analysis_info['formats']
foveal_bound = amblyo_analysis_info['foveal_bound']
rois = amblyo_analysis_info['rois']

# load control settings
with open('{}/{}/analysis_code/settings.json'.format(code_dir, control_dir)) as f:
    json_s = f.read()
    control_analysis_info = json.loads(json_s)
control_subjects = control_analysis_info['subjects']

# define analysis settings
datasets = ['amblyo', 'control']
ecc_categories = ['foveal', 'peripheral']

#### Indivudal analysis
Get for each subject, the weighted average pRF size and CM for foveal and peripheral pRF and save its type

In [25]:
# Amblyopic patients

for format_ in formats:
    print('Format: {}'.format(format_))
    
    for dataset in ['amblyo', 'control']:
        if dataset == 'amblyo':
            dataset_dir = amblyo_dir
            list_subjects = amblyo_subjects
        elif dataset == 'control':
            dataset_dir = control_dir
            list_subjects = control_subjects
        print('Dataset: {}'.format(dataset))
        
        for subject in list_subjects:
            # Load data thresholded
            tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(
                data_dir, dataset_dir, subject, format_)
            tsv_data_th_fn = "{}/{}_prf_violins.tsv".format(tsv_dir, subject)
            data_th = pd.read_table(tsv_data_th_fn, sep="\t")
    
            # Define subject type
            if dataset == 'amblyo':
                if subject in strab_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'strab'
                elif subject in aniso_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'aniso'
                elif subject in mixed_subjects: 
                    subject_type = 'patient'
                    amblyo_type = 'mixed'
            elif dataset == 'control':
                subject_type = 'control'
                amblyo_type = 'control'
            
            # Define pRF eccenticity category (foveal or peripheral)
            def categorize_ecc(value):
                if value <= foveal_bound: return 'foveal'
                else: return 'peripheral'
            data_th['ecc_category'] = data_th['prf_size'].apply(categorize_ecc)
    
            # Determine weighted average and std per subjects of pRF size for fovea and periphery
            def weighted_average(df_groupby, column_data, column_weight):
                return (df_groupby[column_data] * df_groupby[column_weight]).sum() / df_groupby[column_weight].sum()
    
            # Compute table of weighted average by 'roi' and 'ecc_category'
            df_prf_ecc_cat = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_size', 'prf_loo_r2').reset_index(name='prf_size_weighted_avg')
            df_prf_ecc_cat['pcm_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'pcm', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['prf_loo_r2_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_loo_r2', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['prf_size_weighted_avg'] = data_th.groupby(['roi', 'ecc_category']).apply(weighted_average, 'prf_size', 'prf_loo_r2').reset_index()[0]
            df_prf_ecc_cat['vert_area_sum'] = data_th.groupby(['roi', 'ecc_category'])['vert_area'].sum().reset_index()['vert_area']
            df_prf_ecc_cat['subject'] = subject
            df_prf_ecc_cat['subject_type'] = subject_type
            df_prf_ecc_cat['amblyo_type'] = amblyo_type

            # Save tsv
            tsv_prf_ecc_cat_fn = "{}/{}_prf_ecc_cat.tsv".format(tsv_dir, subject)
            print('Saving tsv ({}: {} {}): {}'.format(subject, subject_type, amblyo_type, tsv_prf_ecc_cat_fn))
            df_prf_ecc_cat.to_csv(tsv_prf_ecc_cat_fn, sep="\t", na_rep='NaN', index=False)

Format: fsnative
Dataset: amblyo
Saving tsv (sub-01: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-01/fsnative/prf/tsv/sub-01_prf_ecc_cat.tsv
Saving tsv (sub-02: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-02/fsnative/prf/tsv/sub-02_prf_ecc_cat.tsv
Saving tsv (sub-03: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-03/fsnative/prf/tsv/sub-03_prf_ecc_cat.tsv
Saving tsv (sub-04: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-04/fsnative/prf/tsv/sub-04_prf_ecc_cat.tsv
Saving tsv (sub-05: patient strab): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-05/fsnative/prf/tsv/sub-05_prf_ecc_cat.tsv
Saving tsv (sub-06: patient mixed): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-06/fsnative/prf/tsv/sub-06_prf_ecc_cat.tsv
Saving tsv (sub-07: patient aniso): /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/

In [202]:
#### Group analysis
# get data of all subjects and combined them
for format_ in formats:
    print('Format: {}'.format(format_))
    for j, dataset in enumerate(datasets):
        if dataset == 'amblyo':
            dataset_dir = amblyo_dir
            list_subjects = amblyo_subjects
        elif dataset == 'control':
            dataset_dir = control_dir
            list_subjects = control_subjects
        
        for i, subject in enumerate(list_subjects):
            
            # Load data thresholded
            tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(
                data_dir, dataset_dir, subject, format_)
            tsv_prf_ecc_cat_fn = "{}/{}_prf_ecc_cat.tsv".format(tsv_dir, subject)
            df_prf_ecc_cat_indiv = pd.read_table(tsv_prf_ecc_cat_fn, sep="\t")

            print('Adding {} {}: {}'.format(dataset, subject, tsv_prf_ecc_cat_fn))
            if i == 0 and j==0 :df_prf_ecc_cat = df_prf_ecc_cat_indiv.copy()
            else: df_prf_ecc_cat = pd.concat([df_prf_ecc_cat, df_prf_ecc_cat_indiv])

    # Subject type: control vs. patient
    df_prf_ecc_cat_subject_type_mean = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'subject_type', ]).mean()
    df_prf_ecc_cat_subject_type_mean.columns = ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean', 'vert_area_sum_mean']
    df_prf_ecc_cat_subject_type_sem = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'subject_type', ]).sem()
    df_prf_ecc_cat_subject_type_sem.columns = ['prf_size_weighted_avg_sem', 'pcm_weighted_avg_sem', 'prf_loo_r2_weighted_avg_sem', 'vert_area_sum_sem']
    df_prf_ecc_cat_subject_type = pd.concat([df_prf_ecc_cat_subject_type_mean, df_prf_ecc_cat_subject_type_sem], axis=1).reset_index()

    # Amblyo type: n/a vs. aniso vs. strab vs. mixed
    df_prf_ecc_cat_amblyo_type_mean = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'amblyo_type', ]).mean()
    df_prf_ecc_cat_amblyo_type_mean.columns = ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean', 'vert_area_sum_mean']
    df_prf_ecc_cat_amblyo_type_sem = df_prf_ecc_cat.groupby(['roi', 'ecc_category', 'amblyo_type', ]).sem()
    df_prf_ecc_cat_amblyo_type_sem.columns = ['prf_size_weighted_avg_sem', 'pcm_weighted_avg_sem', 'prf_loo_r2_weighted_avg_sem', 'vert_area_sum_sem']
    df_prf_ecc_cat_amblyo_type = pd.concat([df_prf_ecc_cat_amblyo_type_mean, df_prf_ecc_cat_amblyo_type_sem], axis=1).reset_index()
    

    # Difference by subject_type
    df_subject_type = df_prf_ecc_cat_subject_type
    df_amblyo_type = df_prf_ecc_cat_amblyo_type
    for i, roi in enumerate(rois):
        for j, ecc_category in enumerate(ecc_categories):

            df_patient = df_subject_type.loc[(df_subject_type.roi==roi) & (df_subject_type.subject_type=='patient') & (df_subject_type.ecc_category==ecc_category)]
            df_control = df_subject_type.loc[(df_subject_type.roi==roi) & (df_subject_type.subject_type=='control') & (df_subject_type.ecc_category==ecc_category)]
            
            # patient - control
            df_subject_type_diff_line = pd.DataFrame()
            df_subject_type_diff_line['roi'] = [roi]
            df_subject_type_diff_line['diff_type'] = ['[patient-control]']
            df_subject_type_diff_line['ecc_category'] = [ecc_category]
            for column_name in ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean','vert_area_sum_mean']:
                df_subject_type_diff_line[column_name] = np.array(df_patient[column_name])-np.array(df_control[column_name])

            if i == 0 and j==0: df_subject_type_diff = df_subject_type_diff_line.copy()
            else: df_subject_type_diff = pd.concat([df_subject_type_diff, df_subject_type_diff_line])

            df_aniso = df_amblyo_type.loc[(df_amblyo_type.roi==roi) & (df_amblyo_type.amblyo_type=='aniso') & (df_amblyo_type.ecc_category==ecc_category)]
            df_strab = df_amblyo_type.loc[(df_amblyo_type.roi==roi) & (df_amblyo_type.amblyo_type=='strab') & (df_amblyo_type.ecc_category==ecc_category)]
            df_mixed = df_amblyo_type.loc[(df_amblyo_type.roi==roi) & (df_amblyo_type.amblyo_type=='mixed') & (df_amblyo_type.ecc_category==ecc_category)]
            df_control = df_amblyo_type.loc[(df_amblyo_type.roi==roi) & (df_amblyo_type.amblyo_type=='control') & (df_amblyo_type.ecc_category==ecc_category)]
            
            # aniso / strab / mixed - control
            df_amblyo_type_diff_line = pd.DataFrame()
            df_amblyo_type_diff_line['roi'] = [roi]
            df_amblyo_type_diff_line['diff_type'] = ['[patient-control]']
            df_amblyo_type_diff_line['ecc_category'] = [ecc_category]
            for column_name in ['prf_size_weighted_avg_mean', 'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean','vert_area_sum_mean']:
                df_amblyo_type_diff_line[column_name] = np.array(df_patient[column_name]) - np.array(df_control[column_name])

            if i == 0 and j==0: df_amblyo_type_diff = df_amblyo_type_diff_line.copy()
            else: df_amblyo_type_diff = pd.concat([df_amblyo_type_diff, df_amblyo_type_diff_line])
    debdd
                
    # save per format in amblyoprf

Format: fsnative
Adding amblyo sub-01: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-01/fsnative/prf/tsv/sub-01_prf_ecc_cat.tsv
Adding amblyo sub-02: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-02/fsnative/prf/tsv/sub-02_prf_ecc_cat.tsv
Adding amblyo sub-03: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-03/fsnative/prf/tsv/sub-03_prf_ecc_cat.tsv
Adding amblyo sub-04: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-04/fsnative/prf/tsv/sub-04_prf_ecc_cat.tsv
Adding amblyo sub-05: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-05/fsnative/prf/tsv/sub-05_prf_ecc_cat.tsv
Adding amblyo sub-06: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-06/fsnative/prf/tsv/sub-06_prf_ecc_cat.tsv
Adding amblyo sub-07: /home/mszinte/disks/meso_S/data/amblyo_prf/derivatives/pp_data/sub-07/fsnative/prf/tsv/sub-07_prf_ecc_cat.tsv
Adding amblyo sub-08: /home/mszinte/disks/meso_S/data/ambly

NameError: name 'debdd' is not defined

In [203]:
df_subject_type_diff

,roi,diff,ecc_category,prf_size_weighted_avg_mean,pcm_weighted_avg_mean,prf_loo_r2_weighted_avg_mean,vert_area_sum_mean
0,V1,[patient-control],foveal,0.150643,-7.311829,-0.257006,-1376.568616
0,V1,[patient-control],peripheral,1.200220,0.855558,-0.044851,364.531351
0,V2,[patient-control],foveal,0.350034,-9.090786,-0.233353,-1461.799663
0,V2,[patient-control],peripheral,1.617874,0.104197,0.019830,473.360419
0,V3,[patient-control],foveal,0.150000,-8.087933,-0.249680,-1227.482938
0,V3,[patient-control],peripheral,2.322161,-0.942011,-0.031669,412.132280
0,V3AB,[patient-control],foveal,-0.072650,-6.274743,-0.275873,-577.487759
0,V3AB,[patient-control],peripheral,2.433778,-3.372437,-0.188669,-330.114152
0,LO,[patient-control],foveal,-0.287795,-6.339562,-0.195124,-542.377442
0,LO,[patient-control],peripheral,0.904559,-2.053530,-0.156728,-833.305489


In [197]:
np.array(df_patient['prf_size_weighted_avg_mean'])-np.array(df_control['prf_size_weighted_avg_mean'])

array([0.150643])

In [199]:
df_subject_type.columns

Index(['roi', 'ecc_category', 'subject_type', 'prf_size_weighted_avg_mean',
       'pcm_weighted_avg_mean', 'prf_loo_r2_weighted_avg_mean',
       'vert_area_sum_mean', 'prf_size_weighted_avg_sem',
       'pcm_weighted_avg_sem', 'prf_loo_r2_weighted_avg_sem',
       'vert_area_sum_sem'],
      dtype='object')